# Flea Circus - Problem 213
<p>A $30 \times 30$ grid of squares contains $900$ fleas, initially one flea per square.<br>
When a bell is rung, each flea jumps to an adjacent square at random (usually $4$ possibilities, except for fleas on the edge of the grid or at the corners).</p>

<p>What is the expected number of unoccupied squares after $50$ rings of the bell? Give your answer rounded to six decimal places.</p>

## Solution.

In [43]:
from functools import lru_cache

In [111]:
@cache
def neighbourhood(i, j, n):
    ''' Returns the neighbourhood of (i, j) cell'''
    ans = []
    if i+1 <= n:
        ans.append([i+1, j])
    if i-1 >= 1:
        ans.append([i-1, j])
    if j+1 <= n:
        ans.append([i, j+1])
    if j-1 >= 1:
        ans.append([i, j-1])
    return ans


@cache
def number_of_possibilities(i, j, n):
    ''' Return number of possible jums from the (i,j)-th square'''
    if (i, j) == (1, 1) or (i, j) == (1, n) or (i, j) == (n, 1) or (i, j) == (n, n):
        return 2

    if i == 1 or i == n or j == 1 or j == n:
        return 3

    return 4

In [75]:
@cache
def P(x, y, i, j, n, k):
    ''' Probability that the flea that started from the (x, y) square is on (i,j)-th square after k rings of the bell on nxn board'''
    if k == 0:
        return (x,y) == (i,j)

    if abs(i-x) + abs(j-y) > k:
        return 0

    V = neighbourhood(i, j, n)
    ans = 0
    for v in V:
        vx, vy = v
        ans += P(x, y, vx, vy, n, k-1) * 1/number_of_possibilities(vx, vy, n)
    return ans   

    

@cache
def p(i, j, n, k):
    ''' Probability that the (i,j)-th square is unoccupied after k rings of the bell on nxn board'''
    ans = 1
    for x in range(1, n+1):
        for y in range(1, n+1):
            ans *= (1-P(x, y, i, j, n, k))
    return ans

In [63]:
def e(n, k):
    ''' Solves the question for nxn board after k rings of the bell'''
    ans = sum([p(i, j, n, k) for i in range(1, n+1) for j in range(1, n+1)])

    return round(ans, 6)

In [78]:
e(30, 50)

330.721154

### Markov chains

In [149]:
import numpy as np

In [200]:
def sol213(n, k):
    M = []
    for _ in range(n**2):
        M.append([0]*n**2)
        
    for i in range(1, n+1):
        for j in range(1, n+1):
            V = neighbourhood(i, j, n)
            for v in V:
                vx, vy = v
                M[(i-1)*n+j-1][(vx-1)*n+vy-1] = (1/len(V))

    T = np.array(M)
    T_k = np.linalg.matrix_power(T, k)

    p = np.ones(n**2)
    for i in range(1, n+1):
        for j in range(1, n+1):
            x = np.zeros(n**2)
            x[(i-1)*n + j-1] = 1
            p *= (1 - x @ T_k)

    return round(np.sum(p), 6)

In [201]:
sol213(30, 50)

np.float64(330.721154)